In [1]:
import resnet20
import lr_scheduler

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

ModuleNotFoundError: No module named 'resnet20'

In [ ]:
AUTO = tf.data.AUTOTUNE
CROP_TO = 32
SEED = 42

PROJECT_DIM = 2048
BATCH_SIZE = 64
EPOCHS = 100     

In [3]:
#### Data Augmentation

def add_noise_1(joint_states):
    """Thêm bớt giá trị ngẫu nhiên trong khoảng ±1% giá trị của từng phần tử."""
    noise = tf.random.uniform(shape=tf.shape(joint_states), minval=-0.01, maxval=0.01)
    return joint_states * (1 + noise)

def add_noise_2(joint_states):
    """Thêm bớt giá trị ngẫu nhiên trong khoảng ±2% giá trị của từng phần tử."""
    noise = tf.random.uniform(shape=tf.shape(joint_states), minval=-0.02, maxval=0.02)
    return joint_states * (1 + noise)

In [ ]:
# def gaussian_blur(image, kernel_size=5, sigma=1.0):
#     """Apply Gaussian Blur lên ảnh."""
#     image = tfa.image.gaussian_filter2d(image, filter_shape=(kernel_size, kernel_size), sigma=sigma)
#     return image

# def random_crop(image, crop_size=(100, 100)):
#     """Random cropping ảnh."""
#     cropped_image = tf.image.random_crop(image, size=(crop_size[0], crop_size[1], 3))
#     # Resize lại về kích thước ban đầu
#     return tf.image.resize(cropped_image, size=(128, 128))

# def augmentation_2(image):
#     """Apply Gaussian Blur hoặc Random Cropping."""
#     # Chọn Gaussian Blur hoặc Random Cropping ngẫu nhiên
#     return tf.cond(
#         tf.random.uniform([]) > 0.5,
#         lambda: gaussian_blur(image),
#         lambda: random_crop(image)
#     )

# def augmentation_1(image):
#     """Hàm augment phiên bản 1."""
#     image = tf.image.random_flip_left_right(image)
#     image = tf.image.random_brightness(image, max_delta=0.1)
#     return image

# def augmentation_2(image):
#     """Hàm augment phiên bản 2."""
#     image = tf.image.random_flip_up_down(image)
#     image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
#     return image

# Hàm thêm noise vào joint states
# def add_noise_1(joint_states):
#     """Hàm noise phiên bản 1."""
#     noise = tf.random.normal(shape=tf.shape(joint_states), mean=0.0, stddev=0.01)
#     return joint_states + noise

# def add_noise_2(joint_states):
#     """Hàm noise phiên bản 2."""
#     noise = tf.random.normal(shape=tf.shape(joint_states), mean=0.0, stddev=0.02)
#     return joint_states + noise


In [5]:
#### Data Loader

# Custom augmentation pipelines
def preprocess_1(image, joint_states):
    """Biến đổi phiên bản 1."""
    return augmentation_1(image), add_noise_1(joint_states)

def preprocess_2(image, joint_states):
    """Biến đổi phiên bản 2."""
    return augmentation_2(image), add_noise_2(joint_states)

# Tạo DataLoaders
def create_dataloader(image_data, joint_states_data, batch_size):
    """
    Tạo hai DataLoaders riêng biệt với augmentation_1 và augmentation_2.
    """
    # Chuyển đổi dữ liệu thành TensorFlow tensors
    image_data = tf.convert_to_tensor(image_data, dtype=tf.float32)
    joint_states_data = tf.convert_to_tensor(joint_states_data, dtype=tf.float32)

    # Dataset cho phiên bản 1
    dataset_one = (
        tf.data.Dataset.from_tensor_slices((image_data, joint_states_data))
        .shuffle(1024)
        .map(preprocess_1, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    # Dataset cho phiên bản 2
    dataset_two = (
        tf.data.Dataset.from_tensor_slices((image_data, joint_states_data))
        .shuffle(1024)
        .map(preprocess_2, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    # Zip hai datasets lại
    combined_dataset = tf.data.Dataset.zip((dataset_one, dataset_two))

    return combined_dataset

In [6]:
# Dữ liệu giả lập
num_samples = 700
image_data = np.random.rand(num_samples, 128, 128, 3).astype(np.float32)  # 700 ảnh RGB
joint_states_data = np.random.rand(num_samples, 3).astype(np.float32)  # 700 vector joint states (3 features)

# Tạo DataLoader
batch_size = 32
ssl_dataset = create_dataloader(image_data, joint_states_data, batch_size)

# Duyệt qua các batch để kiểm tra
for (batch_one, batch_two) in ssl_dataset:
    image_a, joint_a = batch_one
    image_b, joint_b = batch_two

    print(f"Image A batch shape: {image_a.shape}")  # (batch_size, 128, 128, 3)
    print(f"Joint A batch shape: {joint_a.shape}")  # (batch_size, 3)
    print(f"Image B batch shape: {image_b.shape}")  # (batch_size, 128, 128, 3)
    print(f"Joint B batch shape: {joint_b.shape}")  # (batch_size, 3)
    break


ValueError: in user code:

    File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_6588\4006438974.py", line 10, in preprocess_2  *
        return augmentation_2(image), add_noise_2(joint_states)
    File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_6588\1086932100.py", line 71, in augmentation_2  *
        lambda: random_crop(image)
    File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_6588\1086932100.py", line 52, in gaussian_blur  *
        blurred = tf.nn.depthwise_conv2d(image, kernel, strides=[1, 1, 1, 1], padding='SAME')

    ValueError: Dimensions must be equal, but are 3 and 1 for '{{node cond/depthwise}} = DepthwiseConv2dNative[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="SAME", strides=[1, 1, 1, 1]](cond/ExpandDims_2, cond/ExpandDims_1)' with input shapes: [1,128,128,3], [5,5,1,1].


In [ ]:
#### Barlow Twin